In [ ]:
import numpy as np
np.random.seed(2016)

import os
import glob
import cv2
import math
import pickle
import datetime
import pandas as pd
#import statistics
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from keras.models import Sequential
from keras.applications import VGG16,VGG19
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.optimizers import SGD
from keras.utils import np_utils
from keras.models import model_from_json
from sklearn.metrics import log_loss
from scipy.misc import imread, imresize

In [ ]:
def get_im_cv2(path, img_rows, img_cols, color_type=3):
    # Load as grayscale
    if color_type == 1:
        img = cv2.imread(path, 0)
    elif color_type == 3:
        img = cv2.imread(path)
    # Reduce size
    resized = cv2.resize(img, (img_cols, img_rows))
    return resized

In [ ]:
def load_train(img_rows, img_cols, color_type=3,num_files_to_read=-1):
    X_train = []
    y_train = []
    
    print('Read train images')
    for j,target_type in enumerate['cat','dog']:
        counter = 0
        print('Load folder Type_{}'.format(j))
        path = os.path.join('..', 'input', 'train',target_type+'*.jpg')
        files = glob.glob(path)
        for fl in files:
            flbase = os.path.basename(fl)
            img = get_im_cv2(fl, img_rows, img_cols, color_type)
            X_train.append(np.asarray(img))
            y_train.append(j)
            counter+=1
            if (counter>=num_files_to_read)&(num_files_to_read>0):
                break
    

    return np.array(X_train), np.array(y_train)

In [ ]:
def cache_data(data, path):
    if os.path.isdir(os.path.dirname(path)):
        file = open(path, 'wb')
        pickle.dump(data, file)
        file.close()
    else:
        print('Directory doesnt exists')

In [ ]:
def restore_data(path):
    data = dict()
    if os.path.isfile(path):
        file = open(path, 'rb')
        data = pickle.load(file)
    return data

In [ ]:
img_rows = 224
img_cols = 224
num_samples = -1
read_from_cache = True
color_type_global = 3
if not read_from_cache:
    X_train, y_train = load_train(img_rows,img_cols,3,num_samples)
    cache_data(X_train,'../processed_input/X_train_{}X{}X{}_{}_max_samples'.format(img_rows,img_cols,
                                                                                   color_type_global,num_samples))
    cache_data(y_train,'../processed_input/y_train_{}_max_samples'.format(num_samples))
else:    
    X_train = restore_data('../processed_input/X_train_{}X{}X{}_{}_max_samples'.format(img_rows,img_cols,
                                                                               color_type_global,num_samples))
    y_train = restore_data('../processed_input/y_train_{}_max_samples'.format(num_samples))

In [ ]:
print X_train.shape
print y_train.shape

In [ ]:
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
from keras.layers import Input, Flatten, Dense
from keras.models import Model

#model = create_model(img_rows, img_cols, color_type_global)
# Generate a model with all layers (with top)
vgg16 = VGG16(weights='imagenet', include_top=True)
model.summary()

In [ ]:
#Add a layer where input is the output of the  second last layer 
x = (vgg16.layers[-4].output)

#Then create the corresponding model 
model = Model(inputs=vgg16.input, outputs=x)
model.summary()

In [ ]:
from keras.utils import to_categorical
from sklearn.metrics import log_loss

sgd = SGD(lr=0.001, decay=1e-5, momentum=0.95, nesterov=True)

model.compile(optimizer=sgd,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# we are not going to train the model so we will not run the next line 
# it is only here for context of regular training / fine tuning of pretrained network
#model.fit(X_train, to_categorical(y_train-1,3),validation_split=0.2,shuffle=True, batch_size=batch_size, epochs=epochs, verbose=1)


In [ ]:
pred_train = model.predict(X_train)
